<a href="https://colab.research.google.com/github/btho733/Belvin-Personal_Projects/blob/master/Project06_MONAIBootcamp/Belvin_WorkingCopy_lab3_networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 3: Networks
---

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Project-MONAI/MONAIBootcamp2020/blob/master/day1notebooks/lab3_networks.ipynb)


### Overview

This notebook introduces you to the MONAI network APIs:
- Convolutions
- Specifying layers with additional arguments
- Flexible definitions of networks

In [1]:
!nvidia-smi

Thu Oct  1 15:32:38 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     8W /  75W |      0MiB /  7611MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Install MONAI and import dependecies
This section installs the latest version of MONAI and validates the install by printing out the configuration.

We'll then import our dependencies and MONAI.  

In [2]:
!pip install -qU "monai[torchvision]==0.3.0rc2"

import torch
import monai
monai.config.print_config()
from monai.networks.layers import Conv
from monai.networks.layers import Act
from monai.networks.layers import split_args
from monai.networks.layers import Pool

     |████████████████████████████████| 296kB 3.3MB/s 
MONAI version: 0.3.0rc2
Python version: 3.6.9 (default, Jul 17 2020, 12:50:27)  [GCC 8.4.0]
Numpy version: 1.18.5
Pytorch version: 1.6.0+cu101

Optional dependencies:
Pytorch Ignite version: NOT INSTALLED or UNKNOWN VERSION.
Nibabel version: 3.0.2
scikit-image version: 0.16.2
Pillow version: 7.0.0
Tensorboard version: 2.3.0
gdown version: 3.6.4
TorchVision version: 0.7.0+cu101
ITK version: NOT INSTALLED or UNKNOWN VERSION.

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



## Unifying the network layer APIs

Network functionality represents a major design opportunity for MONAI. Pytorch is very much unopinionated in how networks are defined. It provides Module as a base class from which to create a network, and a few methods that must be implemented, but there is no prescribed pattern nor much helper functionality for initializing networks. 

This leaves a lot of room for defining some useful 'best practice' patterns for constructing new networks in MONAI. Although trivial, inflexible network implementations are easy enough, we can give users a toolset that makes it much easier to build well-engineered, flexible networks, and demonstrate their value by committing to use them in the networks that we build.

### Convolution as an example

We'll start by taking a look at the Convolution `__doc__` string.

In [3]:
print(Conv.__doc__)

The supported members are: ``CONV``, ``CONVTRANS``.
Please see :py:class:`monai.networks.layers.split_args` for additional args parsing.


The [Conv](https://docs.monai.io/en/latest/networks.html#convolution) class has two options for the first argument. The second argument must be the number of spatial dimensions, `Conv[name, dimension]`, for example:

In [4]:
print(Conv[Conv.CONV, 1])
print(Conv[Conv.CONV, 2])
print(Conv[Conv.CONV, 3])
print(Conv[Conv.CONVTRANS, 1])
print(Conv[Conv.CONVTRANS, 2])
print(Conv[Conv.CONVTRANS, 3])

<class 'torch.nn.modules.conv.Conv1d'>
<class 'torch.nn.modules.conv.Conv2d'>
<class 'torch.nn.modules.conv.Conv3d'>
<class 'torch.nn.modules.conv.ConvTranspose1d'>
<class 'torch.nn.modules.conv.ConvTranspose2d'>
<class 'torch.nn.modules.conv.ConvTranspose3d'>


The configured classes are the "vanilla" PyTorch layers. We could create instances of them by specifying the layer arguments:

In [5]:
print(Conv[Conv.CONV, 2](in_channels=1, out_channels=4, kernel_size=3))
print(Conv[Conv.CONV, 3](in_channels=1, out_channels=4, kernel_size=3))

Conv2d(1, 4, kernel_size=(3, 3), stride=(1, 1))
Conv3d(1, 4, kernel_size=(3, 3, 3), stride=(1, 1, 1))


### Specifying a layer with additional arguments
We'll now take a look at the Activation `__doc__` string.

In [6]:
print(Act.__doc__)

The supported members are: ``ELU``, ``RELU``, ``LEAKYRELU``, ``PRELU``, ``RELU6``, ``SELU``, ``CELU``, ``GELU``, ``SIGMOID``, ``TANH``, ``SOFTMAX``, ``LOGSOFTMAX``.
Please see :py:class:`monai.networks.layers.split_args` for additional args parsing.


The [Act](https://docs.monai.io/en/latest/networks.html#module-monai.networks.layers.Act) classes don't require the spatial dimension information, but supports additional arguments.

In [7]:
print(Act[Act.PRELU])
Act[Act.PRELU](num_parameters=1, init=0.1)

<class 'torch.nn.modules.activation.PReLU'>


PReLU(num_parameters=1)

These could be fully specified with a tuple of `(type_name, arg_dict)`, such as `("prelu", {"num_parameters": 1, "init": 0.1})`:

In [8]:
act_name, act_args = split_args(("prelu", {"num_parameters": 1, "init": 0.1}))
Act[act_name](**act_args)

PReLU(num_parameters=1)

### Putting them together

These APIs allow for flexible definitions of networks.  Below we'll create a class called `MyNetwork` that utilizes `Conv`, `Act`, and `Pool`.  Each Network requires an `__init__` and a `forward` function.

In [9]:
class MyNetwork(torch.nn.Module):

  def __init__(self, dims=3, in_channels=1, out_channels=8, kernel_size=3, pool_kernel=2, act="relu"):
    super(MyNetwork, self).__init__()
    # convolution
    self.conv = Conv[Conv.CONV, dims](in_channels, out_channels, kernel_size=kernel_size)
    # activation
    act_type, act_args = split_args(act)
    self.act = Act[act_type](**act_args)
    # pooling
    self.pool = Pool[Pool.MAX, dims](pool_kernel)
  
  def forward(self, x: torch.Tensor):
    x = self.conv(x)
    x = self.act(x)
    x = self.pool(x)
    return x


This network definition can be instantiated to support either 2D or 3D inputs, with flexible kernel sizes.

It becomes handy when adapting the same architecture design for different tasks,
switching among 2D, 2.5D, 3D easily.

In [10]:
# default network instance
default_net = MyNetwork()
print(default_net)
print(default_net(torch.ones(3, 1, 20, 20, 30)).shape)

# 2D network instance
elu_net = MyNetwork(dims=2, in_channels=3, act=("elu", {"inplace": True}))
print(elu_net)
print(elu_net(torch.ones(3, 3, 24, 24)).shape)

# 3D network instance with anisotropic kernels
sigmoid_net = MyNetwork(3, in_channels=4, kernel_size=(3, 3, 1), act="sigmoid")
print(sigmoid_net)
print(sigmoid_net(torch.ones(3, 4, 30, 30, 5)).shape)

MyNetwork(
  (conv): Conv3d(1, 8, kernel_size=(3, 3, 3), stride=(1, 1, 1))
  (act): ReLU()
  (pool): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)
torch.Size([3, 8, 9, 9, 14])
MyNetwork(
  (conv): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1))
  (act): ELU(alpha=1.0, inplace=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)
torch.Size([3, 8, 11, 11])
MyNetwork(
  (conv): Conv3d(4, 8, kernel_size=(3, 3, 1), stride=(1, 1, 1))
  (act): Sigmoid()
  (pool): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)
torch.Size([3, 8, 14, 14, 2])


Almost all the MONAI layers, blocks and networks are extensions of `torch.nn.modules` and follow this pattern. This makes the implementations compatible with any PyTorch pipelines and flexible with the network design.
The current collections of those differentiable modules are listed in https://docs.monai.io/en/latest/networks.html.

### AHNet

Among those implementations, MONAI features a 3D anisotropic hybrid network (AHNet) with the anisotropic encoder kernels initialised from a pretrained resnet. Please see https://docs.monai.io/en/latest/networks.html#ahnet

## Summary

In this notebook, we recapped MONAI Layers including:
- Convolutions and Activations
- Putting together a base network
- Initialize an AHNet

For full API documentation, please visit https://docs.monai.io.